In [1]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import Tracker
import time

model = YOLO('yolov8s.pt')

def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        colorsBGR = [x, y]
        print(colorsBGR)

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

cap = cv2.VideoCapture('traffic1.mp4')

my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n")

count = 0
tracker = Tracker()
vh_down = {}
counter = []
vh_up = {}
counter1 = []
vh_dtime = {}
vh_utime = {}
cy1 = 322
cy2 = 368
offset = 6

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (1020, 500))

def display_speed(frame, cx, cy, vehicle_id, counter_list, elapsed_time, x, y):
    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
    cv2.putText(frame, str(vehicle_id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

    if vehicle_id not in counter_list:
        counter_list.append(vehicle_id)

    distance = 10

    if elapsed_time > 0:
        speed_ms = distance / elapsed_time
        speed_km = speed_ms * 3.6
        cv2.putText(frame, str(int(speed_km)) + ' Km/h', (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
    else:
        cv2.putText(frame, 'Speed N/A', (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    count += 1
    if count % 3 != 0:
        continue

    frame = cv2.resize(frame, (1020, 500))

    results = model.predict(frame)
    a = results[0].boxes.data
    px = pd.DataFrame(a).astype("float")

    list = []

    for index, row in px.iterrows():
        x1, y1, x2, y2, _, d = map(int, row)
        c = class_list[d]

        if c in ['car', 'truck', 'bus']:
            list.append([x1, y1, x2, y2])

    bbox_id = tracker.update(list)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = (x3 + x4) // 2
        cy = (y3 + y4) // 2

        if cy1 - offset < cy < cy1 + offset:
            vh_down[id] = cy
            vh_dtime[id] = time.time()

        if cy2 - offset < cy < cy2 + offset:
            vh_up[id] = cy
            vh_utime[id] = time.time()

        cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 0, 255), 1)
        if id in vh_down:
            elapsed_time = time.time() - vh_dtime[id]
            display_speed(frame, cx, cy, id, counter, elapsed_time, x4, y4)
        if id in vh_up:
            elapsed_time2 = time.time() - vh_utime[id]
            display_speed(frame, cx, cy, id, counter1, elapsed_time2, x4, y4)

    cv2.line(frame, (267, cy1), (829, cy1), (255, 255, 255), 1)
    cv2.putText(frame, '1line', (274, 318), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

    cv2.line(frame, (167, cy2), (932, cy2), (255, 255, 255), 1)
    cv2.putText(frame, '2line', (181, 363), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

    cv2.putText(frame, 'GoingDown: ' + str(len(counter)), (60, 40), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 0, 0), 2)
    cv2.putText(frame, 'GoingUp: ' + str(len(counter1)), (60, 130), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 0, 0), 2)

    out.write(frame)

    cv2.imshow("RGB", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()


0: 320x640 3 cars, 364.8ms
Speed: 3.0ms preprocess, 364.8ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 335.8ms
Speed: 9.0ms preprocess, 335.8ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 1 truck, 192.9ms
Speed: 2.0ms preprocess, 192.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 5 cars, 1 truck, 188.9ms
Speed: 2.0ms preprocess, 188.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
[783, 269]

0: 320x640 4 cars, 2 trucks, 182.9ms
Speed: 3.0ms preprocess, 182.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)
[919, 488]

0: 320x640 5 cars, 1 truck, 194.9ms
Speed: 2.0ms preprocess, 194.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 1 truck, 194.9ms
Speed: 2.0ms preprocess, 194.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 cars, 1 truck, 187.9ms
Speed: 2.0ms 